<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Trades-Endpoints/blob/main/Query_Order_history_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 查詢訂單歷史紀錄
# 查詢用戶的歷史訂單（訂單狀態為已完成或已取消）。

In [ ]:
# API 使用主要步驟：

# 查詢的最大時間範圍不得超過 7 天
# 預設查詢最近 7 天內的資料
# 回傳的訂單列表依據 updateTime 由小到大排序

In [ ]:
# GET /openApi/swap/v2/trade/allOrders
# API KEY 權限需求：永續合約交易

In [ ]:
# 匯入時間模組
import time
# 匯入requests模組，用於發送HTTP請求
import requests
# 匯入hmac模組，用於產生簽名
import hmac
# 匯入sha256演算法
from hashlib import sha256

In [ ]:
# API主機網址
APIURL = "https://open-api.bingx.com"
# API金鑰（請自行填入）
APIKEY = ""
# SECRET金鑰（請自行填入）
SECRETKEY = ""

In [ ]:
# 範例函式，查詢訂單歷史紀錄
def demo():
    # 請求主體（GET一般為空字典）
    payload = {}
    # API路徑
    path = '/openApi/swap/v2/trade/allOrders'
    # HTTP請求方法
    method = "GET"
    # 查詢歷史訂單參數
    paramsMap = {
        "endTime": "1702731995000",         # 查詢結束時間（毫秒）
        "limit": "500",                     # 查詢筆數限制
        "startTime": "1702688795000",       # 查詢起始時間（毫秒）
        "symbol": "PYTH-USDT",              # 交易對
        "timestamp": "1702731995838"        # 時間戳
    }
    # 組合參數並加上timestamp
    paramsStr = parseParam(paramsMap)
    # 發送API請求
    return send_request(method, path, paramsStr, payload)

In [ ]:
# HMAC-SHA256簽名產生
def get_sign(api_secret, payload):
    # 用hmac和sha256產生簽名
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    # 印出簽名（除錯用）
    print("sign=" + signature)
    # 回傳簽名
    return signature

In [ ]:
# 發送API請求
def send_request(method, path, urlpa, payload):
    # 組合完整請求網址（含簽名）
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    # 印出請求網址
    print(url)
    # 設定HTTP標頭（APIKEY）
    headers = {
        'X-BX-APIKEY': APIKEY,
    }
    # 發送HTTP請求
    response = requests.request(method, url, headers=headers, data=payload)
    # 回傳伺服器回應
    return response.text

In [ ]:
# 將參數字典轉為排序後的查詢字串並加timestamp
def parseParam(paramsMap):
    # 取得排序後的鍵
    sortedKeys = sorted(paramsMap)
    # 依序組合成查詢字串
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])
    # 如果有參數，加上&timestamp
    if paramsStr != "":
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        # 若無其他參數，直接加timestamp
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
# 主程式進入點
if __name__ == '__main__':
    # 執行demo函式並印出回傳內容
    print("demo:", demo())